<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH17_05_%EA%B5%AC%EA%B8%80_BERT%EC%9D%98_%EB%8B%A4%EC%9D%8C_%EB%AC%B8%EC%9E%A5_%EC%98%88%EC%B8%A1(Next_Sentence_Prediction).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.5 MB/s eta 0:00:00


### 1. 다음 문장 예측 모델과 토크나이저
--------------------------

In [2]:
import tensorflow as tf
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer

TFBertForNextSentencePrediction.from_pretrained('모델이름')을 두개 문장이 이어지는 문장 관계인지 여부를 판단하는 BERT 구조를 로드함

AutoTokenizer.from_pretrained('모델이름')을 넣으면 해당 모델이 학습되었을 시 사용했던 토크나이저를 로드함

In [3]:
model = TFBertForNextSentencePrediction.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

All model checkpoint layers were used when initializing TFBertForNextSentencePrediction.

All the layers of TFBertForNextSentencePrediction were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


### 2. BERT 입력
-----------------------------
다음 문장 예측을 위해 문맥 상으로 실제 이어지는 두개 문장을 준비함

In [4]:
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "pizza is eaten with the use of a knife and fork. In casual settings, however, it is cut into wedges to be eaten while held in the hand."

앞서 준비한 토크나이저를 사용하여 두개의 문장을 정수 인코딩함

In [5]:
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

In [6]:
print(encoding['input_ids'])

tf.Tensor(
[[  101  1999  3304  1010 10733  2366  1999  5337 10906  1010  2107  2004
   2012  1037  4825  1010  2003  3591  4895 14540  6610  2094  1012   102
  10733  2003  8828  2007  1996  2224  1997  1037  5442  1998  9292  1012
   1999 10017 10906  1010  2174  1010  2009  2003  3013  2046 17632  2015
   2000  2022  8828  2096  2218  1999  1996  2192  1012   102]], shape=(1, 58), dtype=int32)


여기서 주의할 점은 여기서 101과 102는 특별토큰임. 실제로 해당 토큰나이저의 [CLS] 토큰과 [SEP] 토큰 번호를 출력해 봄

In [7]:
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)
print(tokenizer.sep_token, ':', tokenizer.sep_token_id)

[CLS] : 101
[SEP] : 102


위의 정수 인코딩 결과를 다시 디코딩해 보면 현재 입력의 구성을 확인할 수 있음

In [8]:
print(tokenizer.decode(encoding['input_ids'][0]))

[CLS] in italy, pizza served in formal settings, such as at a restaurant, is presented unsliced. [SEP] pizza is eaten with the use of a knife and fork. in casual settings, however, it is cut into wedges to be eaten while held in the hand. [SEP]


BERT에서 두개 문장이 입력으로 들어갈 경우에는 맨앞에는 [CLS] 토큰이 존재하고, 첫번째 문장이 끝나면 [SEP] 토큰, 그리고 두번째 문장이 종료되었을 때 다시 [SEP] 토큰이 추가됨. 토크나이저로 변환된 결과에서 token_type_ids를 통해 문장을 구분하는 세그먼트 인코딩 결과를 확인할 수 있음

In [9]:
print(encoding['token_type_ids'])

tf.Tensor(
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]], shape=(1, 58), dtype=int32)


### 3. 다음 문장 예측하기
---------------------------
TFBertForNextSentencePrediction을 통해 다음 문장을 예측함. 모델에 입력을 넣으면, 해당 모델은 소프트맥스 함수를 지나기 전 값인 logits을 리턴함. 해당 값을 소프트맥스 함수를 통과시킨 후에 레이블에 대한 확률값을 출력함

In [11]:
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]
softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print(probs)

tf.Tensor([[9.9999714e-01 2.8381912e-06]], shape=(1, 2), dtype=float32)


0번 인덱스에 대한 확률값이 1번 인덱스에 대한 확률값보다 훨씬 큼. 실질적으로 모델이 예측한 레이블은 0이라는 뜻임. 이제 두개의 값 중 더 큰 값을 모델의 예측값으로 판단하도록 더 큰 확률값을 가진 인덱스를 리턴하도록 함

In [15]:
print('최종 예측 레이블:', tf.math.argmax(probs, axis=-1).numpy()[0])

최종 예측 레이블: 0


최종 예측 레이블은 0이며, 이는 BERT가 다음 문장 예측을 학습했을 때 실제 이어지는 두개의 문장 레이블은 0. 이어지지 않는 문장의 경우에는 레이블을 1로 두고 이진 분류로 학습을 하였기 때문임. 


이번에는 이어지지 않는 두개의 문장으로 테스트함함

In [16]:
# 상관없는 두개의 문장 예시
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "The sky is blue due to the shorter wavelength of blue light."

encoding = tokenizer(prompt, next_sentence, return_tensors='tf')
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy()[0])

최종 예측 레이블 : 1
